In [ ]:
import sys, os

AG4MDIR='/kaggle/working/ag4masses'
AGLIB='/kaggle/working/aglib'
AGDIR=f"{AG4MDIR}/alphageometry"
MELIAD_PATH=f"{AGLIB}/meliad"
DATA=f"{AGLIB}/ag_ckpt_vocab"
TESTDIR=f"/kaggle/working/ag4mtest"

# Download Files

In [ ]:
# Run this cell to refresh code and get the latest versions
# AG4MDIR and MELIAD_PATH are in /kaggle/working and will be saved as outputs of the Notebook
# !rm -fr {AG4MDIR} {MELIAD_PATH}

In [ ]:
if not os.path.exists(MELIAD_PATH):
    !git clone https://github.com/google-research/meliad.git {MELIAD_PATH}

if not os.path.exists(AG4MDIR):
    !git clone https://github.com/ZYP123678/alphageometry.git {AG4MDIR}

# Temporarily modified files, upload into dataset tmpfiles
# !cp /kaggle/input/tmpfiles/numericals.py {AGDIR}
# !cp /kaggle/input/tmpfiles/alphageometry.py {AGDIR}

if not os.path.exists(TESTDIR):
    !mkdir {TESTDIR}

In [ ]:
if not os.path.exists(DATA):
  # download data: vocabulary, trained model
  !mkdir {DATA}

  ### Convoluted process. This does no work, seems due to Google Drive restrictions
  # !gdown --folder https://bit.ly/alphageometry/
  #
  ### First got file links from Google Drive web UI, under Share menu. It will download HTML files with download button. Extract URL from the file.
  ### For checkpoint_10999999, because the file is big, there is an additional step asking for confirmation, got the final URL from the 2nd HTML,
  ### Link constructed using AI from HTML form.
  # !gdown https://drive.google.com/file/d/1mRd6J0UkeWoFUjeVB7BQi5lVNLvPBe31/view?usp=drive_link -O {AGLIB}/ag_ckpt_vocab/geometry.757.vocab
  # !gdown https://drive.google.com/file/d/1t-r3KfU8aDbS1UHpdyM3LH21rwSCIXTz/view?usp=drive_link -O {AGLIB}/ag_ckpt_vocab/geometry.757.model
  # !gdown https://drive.google.com/file/d/1qXkmmgoJ8oTYJdFV1xw0xGPpQj6SyOYA/view?usp=drive_link -O {AGLIB}/ag_ckpt_vocab/checkpoint_10999999

  !wget -O {DATA}/geometry.757.vocab "https://drive.usercontent.google.com/uc?id=1mRd6J0UkeWoFUjeVB7BQi5lVNLvPBe31&export=download"
  !wget -O {DATA}/geometry.757.model "https://drive.usercontent.google.com/uc?id=1t-r3KfU8aDbS1UHpdyM3LH21rwSCIXTz&export=download"
  !wget -O {DATA}/checkpoint_10999999 "https://drive.usercontent.google.com/download?id=1qXkmmgoJ8oTYJdFV1xw0xGPpQj6SyOYA&export=download&confirm=t&uuid=ae22f4de-cb77-4145-af5f-8cfbb59e867e"

!ls -l {DATA}

# Setup Env

In [ ]:
# Python packages for AlphaGeometry
!pip cache purge
!pip install --upgrade pip
!pip install --upgrade packaging setuptools setuptools_scm wheel 
!pip install --require-hashes --no-deps -r {AG4MDIR}/alphageometry/requirements.txt

In [ ]:
# Python packages for Nvidia gpu.
# The versions of Python packages used by AlphaGeometry seem to only work with Cuda 11, not 12
!pip install -U "jax==0.4.6" "jaxlib[cuda11_cudnn86]==0.4.6" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install nvidia-cuda-runtime-cu11
!pip install nvidia-pyindex
# !pip list

In [ ]:
# Linux packages for Nvidia gpu.
# The versions of Python packages used by AlphaGeometry seem to only work with Cuda 11, not 12
!apt-get update
!DEBIAN_FRONTEND=noninteractive apt install -y cuda-11-8
# !apt list|grep cuda

In [ ]:
# Information about Nvidia drivers
!nvcc --version
!nvidia-smi

# Execution

In [ ]:
#!! cannot have ' in the script, including in comments
jobScript = '''
# !/bin/bash
set -e
set -x

# stdout, solution is written here
OUTFILE=$TESTDIR/${PROB}.out
# stderr, a lot of information, error message, log etc.
ERRFILE=$TESTDIR/${PROB}.log

# stdout and stderr are written to both ERRFILF and console
exec >$ERRFILE 2>&1

echo PROB=$PROB
echo PROB_FILE=$PROBFILE
echo MODEL=$MODEL

# Directory where output files go
echo TESTDIR=$TESTDIR
# Directory containing AG4Masses source files
echo AG4MDIR=$AG4MDIR
# Directory containing external libraries including ag_ckpt_vocab and meliad
echo AGLIB=$AGLIB

AGDIR=$AG4MDIR/alphageometry
export PYTHONPATH=$PYTHONPATH:$AGDIR:$AGLIB

echo BATCH_SIZE=$BATCH_SIZE
echo BEAM_SIZE=$BEAM_SIZE
echo DEPTH=$DEPTH
echo NWORKERS=$NWORKERS

echo ERRFILE=$ERRFILE
echo OUTFILE=$OUTFILE

DATA=$AGLIB/ag_ckpt_vocab
MELIAD_PATH=$AGLIB/meliad
export PYTHONPATH=$PYTHONPATH:$MELIAD_PATH

# 生成时间戳用于RL数据目录
TIMESTAMP=$(date +"%Y%m%d_%H%M%S")
RL_DATA_DIR=$TESTDIR/rl_data_${TIMESTAMP}

DDAR_ARGS=( \
  --defs_file=$AGDIR/defs.txt \
  --rules_file=$AGDIR/rules.txt \
)

SEARCH_ARGS=(
  --beam_size=$BEAM_SIZE
  --search_depth=$DEPTH
)

# RL数据收集参数
RL_ARGS=(
  --collect_rl_data=$COLLECT_RL_DATA
  --rl_data_dir=$RL_DATA_DIR
)
# 如果不需要收集RL数据，则创建空数组
if [ "$COLLECT_RL_DATA" = "false" ]; then
  RL_ARGS=()
  echo "RL数据收集已禁用"
else
  # 确保RL数据目录存在
  mkdir -p $RL_DATA_DIR
  echo "RL数据将保存到: $RL_DATA_DIR"
fi

LM_ARGS=(
  --ckpt_path=$DATA \
  --vocab_path=$DATA/geometry.757.model \
  --gin_search_paths=$MELIAD_PATH/transformer/configs,$AGDIR \
  --gin_file=base_htrans.gin \
  --gin_file=size/medium_150M.gin \
  --gin_file=options/positions_t5.gin \
  --gin_file=options/lr_cosine_decay.gin \
  --gin_file=options/seq_1024_nocache.gin \
  --gin_file=geometry_150M_generate.gin \
  --gin_param=DecoderOnlyLanguageModelGenerate.output_token_losses=True \
  --gin_param=TransformerTaskConfig.batch_size=$BATCH_SIZE \
  --gin_param=TransformerTaskConfig.sequence_length=128 \
  --gin_param=Trainer.restore_state_variables=False
);

true "=========================================="

python -m alphageometry \\
--alsologtostderr \\
--problems_file=$PROB_FILE \\
--problem_name=$PROB \\
--mode=$MODEL \\
"${DDAR_ARGS[@]}" \\
"${SEARCH_ARGS[@]}" \\
"${RL_ARGS[@]}" \\
"${LM_ARGS[@]}" \\
--out_file=$OUTFILE \\
--n_workers=$NWORKERS 2>&1

# 显示RL数据收集结果
if [ "$COLLECT_RL_DATA" = "true" ]; then
  echo "=========================================="
  echo "RL数据收集完成，保存路径: $RL_DATA_DIR"
  if [ -d "$RL_DATA_DIR" ]; then
    ls -l $RL_DATA_DIR
    COUNT=$(cat $RL_DATA_DIR/*.jsonl 2>/dev/null | wc -l || echo "0")
    echo "共收集了 $COUNT 条样本"
  else
    echo "警告: RL数据目录未创建"
  fi
fi
'''

In [ ]:
os.environ["TESTDIR"]=TESTDIR
os.environ["AG4MDIR"]=AG4MDIR
os.environ["AGLIB"]=AGLIB

# BATCH_SIZE: number of outputs for each LM query
# BEAM_SIZE: size of the breadth-first search queue
# DEPTH: search depth (number of auxilary points to add)
# NWORKERS: number of parallel run worker processes.
# 
# Memory usage is affected by BATCH_SIZE, NWORKER and complexity of the problem.
# Larger NWORKER and BATCH_SIZE tends to cause out of memory issue
#
# The results in Google paper can be obtained by setting BATCH_SIZE=32, BEAM_SIZE=512, DEPTH=16
#
# 1/2025: Kaggle free version provides GPU T4x2, 4 virtual CPUs, 29G RAM. Can set 
#   NWORKERS=2
#   CUDA_VISIBLE_DEVICES=0,1

os.environ["BATCH_SIZE"]="16"
os.environ["BEAM_SIZE"]="64"
os.environ["DEPTH"]="16"
os.environ["NWORKERS"]="2"

# 强化学习数据收集设置
os.environ["COLLECT_RL_DATA"]="true" 

os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

# test problems can be uploaded into a dataset, e.g. for dataset "tmpfiles", "/kaggle/input/tmpfiles/test-problems.txt"
os.environ["PROB_FILE"]=f"{AG4MDIR}/data/ag4m_problems.txt"
PROB="imo-2024-q4"
os.environ["PROB"]=PROB
# alphageometry|ddar
os.environ["MODEL"]="alphageometry"

# In an interactive Kaggle session, run the job in background, so we can do other things in the Notebook.
# For long jobs, commit the Notebook and run in Batch mode.
# An interactive session will be terminated after about 20 minutes of idle time.
if os.environ["KAGGLE_KERNEL_RUN_TYPE"]=="Batch":
    os.system(f"echo '{jobScript}'|bash")
else:
    os.system(f"echo '{jobScript}'|bash &")


In [ ]:
#!cat /kaggle/input/tmpfiles/test-problems.txt

In [ ]:
# In an interactive Kaggle session, run this to see the log file. We can cancel this cell's execution
# to do other things in the Notebook
if os.environ["KAGGLE_KERNEL_RUN_TYPE"] != "Batch":
    !tail -f {TESTDIR}/{PROB}.log

In [ ]:
# Command to kill the background job in an interactive session
# !pkill -P `ps -ef|grep -- '-m alphageometry'|grep -v grep|awk '{printf "%d,%d", $2, $3}'`

# Command to check processes
### |cat to show full commandline
# !ps -eo pid,ppid,pgid,%cpu,cmd |cat
# !ps -ef|cat

In [ ]:
# Command to check progress of a running job in an interactive session
# !bash {AG4MDIR}/utils/checkprog.sh {TESTDIR}/{PROB}.log

In [ ]:
# 查看RL数据收集结果
!find {TESTDIR} -name "rl_data_*" -type d | sort -r | head -1 | xargs ls -la

In [ ]:
# In Batch run, after the job completes, list output files
!ls -ltr {TESTDIR}